In [1]:
%matplotlib widget
from nn import NN
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import timedelta
from sklearn.preprocessing import RobustScaler
# Technical Analysis library
import ta
# Neural Network library
from keras import Model
from keras.models import Sequential, clone_model
from keras.layers import LSTM, Dense, Dropout, Input, Reshape, Concatenate

In [2]:
from keras_visualizer import visualizer 

In [3]:
# # Loading in the Data
# import os, sys
# sys.path.append("../stock_env/")
# from account import Account
# acc = Account("../stock_env/stock_info/")
nn = NN()
df = pd.read_csv("../data/AAPL.csv", date_parser=True)
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,1980-12-12,0.128348,0.128906,0.128348,0.128348,0.100922,469033600
1,1980-12-15,0.122210,0.122210,0.121652,0.121652,0.095657,175884800
2,1980-12-16,0.113281,0.113281,0.112723,0.112723,0.088636,105728000
3,1980-12-17,0.115513,0.116071,0.115513,0.115513,0.090830,86441600
4,1980-12-18,0.118862,0.119420,0.118862,0.118862,0.093463,73449600


In [4]:
def data_by_date(df, start_date=None, end_date=None):
    if start_date == None:
        start_date = df.Date[0]
    if end_date == None:
        end_date = df.Date[-1]
    after_start_date = df["Date"] >= start_date
    before_end_date = df["Date"] <= end_date
    between_two_dates = after_start_date & before_end_date
    filtered_dates = df.loc[between_two_dates]
    return filtered_dates.copy()

In [5]:
df_f = data_by_date(df, end_date="2020-06-01")

In [6]:
nn.preprocess(df_f)
# Splitting the data into appropriate sequences
x, y = nn.split_sequence(nn.df.to_numpy())

/home/jl057/.local/lib/python3.7/site-packages/ta/trend.py:768: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i] / self._trs[i])
/home/jl057/.local/lib/python3.7/site-packages/ta/trend.py:772: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i] / self._trs[i])


1. Model: DRNN
2. Loss: The Huber loss function weighs outliers less than the Mean Squared Error loss function.
3. Optimizer: Adam

In [7]:
# Using DRNN Model (Stacked LSTM Arc)
nn.build_drnn()

In [13]:
# Using Simple LSTM Model
nn.build_lstm()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_7 (LSTM)                (None, 90, 90)            63000     
_________________________________________________________________
dropout_5 (Dropout)          (None, 90, 90)            0         
_________________________________________________________________
lstm_8 (LSTM)                (None, 60)                36240     
_________________________________________________________________
dense_1 (Dense)              (None, 30)                1830      
Total params: 101,070
Trainable params: 101,070
Non-trainable params: 0
_________________________________________________________________


In [14]:
result = nn.model.fit(x, y, epochs=50, batch_size=128, validation_split=0.1, verbose=0)

In [15]:
nn.visualize_training_results(result)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
nn.prediction_vs_actual()
nn.forecast_future()

RMSE: 1.9394972272668416


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
nn.model.save('pred_lstm')

INFO:tensorflow:Assets written to: pred_lstm/assets
